<a href="REF: https://github.com/GoogleCloudPlatform/df-ml-anomaly-detection/blob/master/README.md">Reference1<a/>  <br>
<a href="https://faker.readthedocs.io/en/master/">Reference2<a/> 

In [20]:
import torch
import json
import torch
import typing
from torch import nn
from IPython.display import clear_output
import apache_beam as beam
from datetime import datetime
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import Dataset

In [ ]:
# !pip install datasets gcsfs torch 

In [15]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data.iloc[index, :]
        return sample

In [16]:
class Autoencoder(nn.Module):  # 65-32-8-32-65
    def __init__(self):
        pass

    def encode(self, x):  # 65-32-8
        pass

    def decode(self, x):  # 8-32-65
        pass
        
    def forward(self, x):  # 65-32-8-32-65
        pass


In [57]:
data_files = {"train":"gs://electric-armor-395015-netlog-bucket/aggregate/-2023-10-16T10:24:00-2023-10-16T10:25:00-00000-of-00001"}
netlogdata = load_dataset("json", data_files=data_files)#streaming=True)
clear_output()

In [ ]:
python src/feature-pipeline/feature.py --bucket gs://electric-armor-395015-netlog-bucket --file_name_suffix ".json" --netlog_bq_table "electric-armor-395015.netlog_dataset.log" --input_file_pattern '/anomaly/*.avro' --temp_location 'gs://electric-armor-395015-netlog-bucket/temp'

In [58]:
netlogdata

DatasetDict({
    train: Dataset({
        features: ['subscriberId', 'dstIP', 'UniqueIPs', 'UniquePorts', 'NumRecords', 'MinTxBytes', 'MaxTxBytes', 'AvgTxBytes', 'MinRxBytes', 'MaxRxBytes', 'AvgRxBytes', 'MinDuration', 'MaxDuration', 'AvgDuration'],
        num_rows: 511
    })
})

In [59]:
from apache_beam.internal import pickler
pickler.set_library(pickler.USE_CLOUDPICKLE)

In [51]:
train = netlogdata["train"]

In [55]:
# shuffled_dataset = train.take(5)
next(iter(shuffled_dataset))

{'subscriberId': 'e5bfd709-3e13-4fff-a4e3-7a84f9cdcc0a',
 'dstIP': '10.235.124.231',
 'UniqueIPs': 1,
 'UniquePorts': 1,
 'NumRecords': 22,
 'MinTxBytes': 68,
 'MaxTxBytes': 704,
 'AvgTxBytes': 296.4545454545,
 'MinRxBytes': 62,
 'MaxRxBytes': 466,
 'AvgRxBytes': 233.5,
 'MinDuration': 1,
 'MaxDuration': 10,
 'AvgDuration': 5.1363636364}

In [ ]:
!docker build -t hardeybisey/netlog-event-generator:latest .
!docker push hardeybisey/netlog-event-generator:latest
!python main.py --runner=DataflowRunner --project=electric-armor-395015 --region=europe-west2 --temp_location=gs://electric-armor-395015-netlog-bucket/tmp --sdk_container_image=hardeybisey/netlog-event-generator --topic=projects/electric-armor-395015/topics/netlog-topic --qps=1000 --event_type=anomaly --sdk_location=container --pickle_library=cloudpickle

In [127]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [157]:
a = b"{\"name\": \"Adam\", \"age\": 30}"
a.update

AttributeError: 'bytes' object has no attribute 'update'

In [152]:
reader = DataFileReader(open("/Users/hardey/Downloads/raw_2023-10-16T10 34 20+00 00_f4cc9c.avro", "rb"), DatumReader())
for user in reader:
    break
print(user)
reader.close()

{'subscription_name': 'gsc-sub', 'message_id': '8886770784205506', 'publish_time': datetime.datetime(2023, 10, 16, 10, 34, 20, 118000, tzinfo=<avro.timezones.UTCTzinfo object at 0x7fb92b52ba60>), 'attributes': {}, 'data': b'{"subscriberId": "067b56b3-9cf2-4b20-a695-22e7d21d2198", "srcIP": "23.53.254.31", "srcPort": 2781, "dstIP": "10.104.26.160", "dstPort": 3806, "txBytes": 372, "rxBytes": 275, "startTime": "2023-10-16T10:34:19", "endTime": "2023-10-16T10:34:24", "tcpFlag": "PSH", "protocolName": "TCP", "protocolNumber": 6}'}
